In [ ]:
"""
Полный пайплайн приёма изображений с погодных спутников NOAA
"""

import numpy as np
from scipy import signal
from PIL import Image

class NOAADecoder:
    def __init__(self, sample_rate=2.4e6):
        self.fs = sample_rate
        self.carrier_freq = 2400  # Несущая поднесущей
        
    def capture_noaa_pass(self, frequency=137.62e6, duration=600):
        """
        Захват прохода спутника NOAA
        frequency: 137.62 MHz для NOAA-15/18/19
        duration: ~10 минут (600 секунд)
        """
        print(f"🛰️  Захват NOAA на {frequency/1e6:.2f} МГц...")
        print(f"⏱️  Длительность: {duration} секунд")
        
        # С реальным RTL-SDR:
        # from rtlsdr import RtlSdr
        # sdr = RtlSdr()
        # sdr.sample_rate = self.fs
        # sdr.center_freq = frequency
        # sdr.gain = 40
        # samples = sdr.read_samples(int(self.fs * duration))
        
        # Симуляция для примера
        samples = self.simulate_noaa_signal(duration)
        return samples
    
    def simulate_noaa_signal(self, duration):
        """Симуляция APT сигнала"""
        t = np.arange(0, duration, 1/self.fs)
        
        # APT сигнал (AM с поднесущей 2400 Гц)
        subcarrier = np.cos(2 * np.pi * self.carrier_freq * t)
        
        # Добавляем "изображение" (пример)
        image_freq = 2  # Гц (скорость сканирования)
        modulation = 0.5 * (1 + np.sin(2 * np.pi * image_freq * t))
        
        signal_data = modulation * subcarrier
        noise = 0.1 * np.random.randn(len(t))
        
        return signal_data + noise
    
    def demodulate_am(self, signal_data):
        """AM демодуляция"""
        # Огибающая через преобразование Гильберта
        analytic = signal.hilbert(signal_data)
        envelope = np.abs(analytic)
        return envelope
    
    def decode_apt_image(self, signal_data):
        """Декодирование APT изображения"""
        # 1. Децимация до аудио частоты
        decimation_factor = int(self.fs / 20800)  # 20.8 kHz
        audio = signal.decimate(signal_data, decimation_factor)
        
        # 2. AM демодуляция
        envelope = self.demodulate_am(audio)
        
        # 3. Фильтрация поднесущей (полосовой фильтр 2400 Гц)
        sos = signal.butter(5, [2300, 2500], btype='band', 
                           fs=20800, output='sos')
        filtered = signal.sosfilt(sos, envelope)
        
        # 4. Детектирование огибающей поднесущей
        apt_signal = self.demodulate_am(filtered)
        
        # 5. Ресемплинг до частоты пикселей (4160 пикселей/линия)
        # APT: 2 линии/секунду, 4160 пикселей/линия
        line_rate = 2  # Гц
        pixels_per_line = 4160
        pixel_rate = line_rate * pixels_per_line
        
        # Децимация до частоты пикселей
        resample_factor = 20800 / pixel_rate
        image_data = signal.resample(apt_signal, 
                                     int(len(apt_signal) / resample_factor))
        
        # 6. Формирование изображения
        num_lines = len(image_data) // pixels_per_line
        image_array = image_data[:num_lines * pixels_per_line]
        image_array = image_array.reshape(num_lines, pixels_per_line)
        
        # Нормализация 0-255
        image_array = ((image_array - image_array.min()) / 
                      (image_array.max() - image_array.min()) * 255)
        
        return image_array.astype(np.uint8)
    
    def save_image(self, image_array, filename='noaa_apt.png'):
        """Сохранение изображения"""
        img = Image.fromarray(image_array, mode='L')
        img.save(filename)
        print(f"✅ Изображение сохранено: {filename}")
        return img

# Использование
decoder = NOAADecoder()
signal_data = decoder.capture_noaa_pass(frequency=137.62e6, duration=600)
image = decoder.decode_apt_image(signal_data)
decoder.save_image(image)